In [16]:
import csv
import os
import time
import wave as we
import numpy as np
from sys import argv
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, stats 
from scipy.signal import butter, lfilter

In [17]:
#Filter: Moving Average and High Pass Butter

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def butter_highpass(cutoff, fs, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=6):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [18]:
with open("/Users/lily/Desktop/data3888/data_final/training_set_2.dat") as f:
    x = f.read().splitlines()
frequency = 10000
fs = frequency
length = len(x)
indices = np.arange(0,length,1)                  #TAKES A PORTION OF THE DATA
TAKEN = np.take(x, indices)

string = ",".join(TAKEN)                        #PUTS INTO ONE STRING SEPARATED BY A COMMA BUT STILL WITH SPACES
stringnew = string.replace(" ", "")             #REMOVES SPACES
list = stringnew.split(',')                     #SPLITS SUCH THAT IT IS A LIST WITH STRINGS EG ['796.0', '742.0' ...]
arr = np.array(list)
arr = arr.astype(np.float)                      #TURNS INTO AN ARRAY OF FLOATS

filtered = moving_average(arr,1000)            #MOVING AVERAGE DATA
filtlength = len(filtered)

indices2 = np.arange(1,filtlength+1,1)#UPDATE INDICES FOR MOVING AVERAGE
T = indices2 / frequency

fc = 15  # Cut-off frequency of the filter
w = fc / (fs / 2) # Normalize the frequency
b, a = signal.butter(5, w, 'low')
output= signal.filtfilt(b, a, filtered) 

TWindow = 1.2    # observation interval given in seconds
TMax = np.max(T)  # maximum time recorded
dT = T[1]-T[0] # dT
NWindow = int(round(TWindow/dT))   # number of points in Window
VALUELIST = []
TIMELIST = []

for k in range(0,int(round(TMax/TWindow))): #for loop with how many windows fit in time in this case its 240 (#2)

    if k == 0:
        noise_level = np.mean(output[0:NWindow])
        threshold_level = 1.05 * noise_level    #arbitrary value I picked
        thresh=threshold_level - noise_level


    window_index_beginning = 1+NWindow*(k-1)
    window_index_end = NWindow+NWindow*(k-1)
    window_index_range = range(window_index_beginning, window_index_end)        #MAIN WINDOW PARAMETERS
    indices = np.arange(window_index_beginning, window_index_end+1, 1)

    Tlil = indices / frequency                                                  #TIME IN WINDOW
    array = np.take(output,indices)                                             #VALUES IN WINDOW


    maxwindow = np.max(array)
    maxindi = np.where(array == maxwindow)
    maxT = Tlil[maxindi[0]]   #GIVES TIME FOR MAX VALUE
    minwindow = np.min(array)
    window_val_range = maxwindow - minwindow

    if window_val_range > thresh*1.5:
        #print("got window",k)
        array2 = array - noise_level
        maxwindow = abs(np.max(array2)) #can find abs value of greatest deviation from noiselevel
        maxindi = np.where(array2 == maxwindow)
        maxT = Tlil[maxindi[0]]   #where abs(max) occurs in unit time
        indexT = Tlil*frequency
        # print(indexT)

        new_window_index_beginning = maxindi[0] - NWindow
        new_window_index_end = maxindi[0] + NWindow
        new_window_index_range = range(window_index_beginning, window_index_end)  # WINDOW PARAMETERS
        new_indices = np.arange(window_index_beginning, window_index_end + 1, 1)

        new_T = new_indices / frequency
        new_array = np.take(output, new_indices)
        #print(new_T)  #SHOWS NEW WINDOW IN SECONDS
        

fig = plt.figure(figsize=(50, 30))             #PLOT MOVING AVERAGE
plt.plot(T,output)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lily/Desktop/data3888/data_final/training_set_2.dat'

In [22]:
#checking noise ratio 
with open("/Users/lily/Desktop/data3888/filtereddata/training_2.dat") as f:
    x = f.read().splitlines()
    x = np.array(x).astype(np.float)

def signaltonoise(data, axis=0, ddof=0):
    data = np.asanyarray(data)
    m = data.mean(axis)
    sd = data.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)


noise_ratio = signaltonoise(x, axis=0, ddof=0)
print(noise_ratio)

69.6459632522992


In [200]:
#SD of left
i=1
left_total = []
while i < 11:
    filename = 'filtereddata/left'+str(i)+'.dat'
    array = np.loadtxt(filename)
    var_left = np.var(array)
    left_total.append(np.sqrt(var_left))
    i+=1
# range
left_max_sd = max(left_total)
left_min_sd = min(left_total)
print(left_max_sd)
print(left_min_sd)
# mean of SD
left_mean_SD = np.mean(left_total)
print(left_mean_SD)
# sd of sd 
left_sd_SD = np.sqrt(np.var(left_total))
print(left_sd_SD)


10.16018782761592
6.925735922458598
8.18062370751979
1.030337923560787


In [201]:
#SD of blink
i=1
blink_total = []
while i < 11:
    filename = 'filtereddata/blink'+str(i)+'.dat'
    array = np.loadtxt(filename)
    
    var_blink = np.var(array)
    blink_total.append(np.sqrt(var_blink))
    i+=1
# range
blink_max_sd = max(blink_total)
blink_min_sd = min(blink_total)
print(blink_max_sd)
print(blink_min_sd)
# mean of SD
blink_mean_SD = np.mean(blink_total)
print(blink_mean_SD)
# sd of sd 
blink_sd_SD = np.sqrt(np.var(blink_total))
print(blink_sd_SD)

6.110328941578203
2.605670853601645
4.242351315509719
1.1336186829861947


In [202]:
#SD of right
i=1
right_total = []
while i < 11:
    filename = 'filtereddata/right'+str(i)+'.dat'
    array = np.loadtxt(filename)
    var_right = np.var(array)
    right_total.append(np.sqrt(var_right))
    i+=1
# range
right_max_sd = max(right_total)
right_min_sd = min(right_total)
print(right_max_sd)
print(right_min_sd)
    
# mean of SD
right_mean_SD = np.mean(right_total)
print(right_mean_SD)
# sd of sd 
right_sd_SD = np.sqrt(np.var(right_total))
print(right_sd_SD)

17.60371830023415
10.698615265392547
12.345555719854904
1.8942559495295712


In [203]:
#SD of doubleblink
i=1
doubleblink_total = []
while i < 11:
    filename = 'filtereddata/doubleblink'+str(i)+'.dat'
    array = np.loadtxt(filename)
    var_doubleblink = np.var(array)
    doubleblink_total.append(np.sqrt(var_doubleblink))
    i+=1
# range
doubleblink_max_sd = max(doubleblink_total)
doubleblink_min_sd = min(doubleblink_total)
print(doubleblink_max_sd)
print(doubleblink_min_sd)

# mean of SD
doubleblink_mean_SD = np.mean(doubleblink_total)
print(doubleblink_mean_SD)
# sd of sd 
doubleblink_sd_SD = np.sqrt(np.var(doubleblink_total))
print(doubleblink_sd_SD)

51.22539870061582
4.046691058945064
27.324143414112104
16.95505596917002


In [209]:
iSampleRate = 10000             
x_left = np.fromfile("filtereddata/left3.dat",dtype=np.float32)
iSampleCount_left = x_left.shape[0]           
t_left = np.linspace(0,iSampleCount_left/iSampleRate,iSampleCount_left)

xFFT_left = np.abs(np.fft.rfft(x_left)/iSampleCount_left)  
xFreqs_left = np.linspace(0, iSampleRate/2, int(iSampleCount_left/2)+1)

print(xFreqs_left)
print(xFFT_left)

[0.00000000e+00 3.26592465e-02 6.53184930e-02 ... 4.99993468e+03
 4.99996734e+03 5.00000000e+03]
[2.13279118e+21 4.75462134e+17 3.24242193e+17 ... 3.83730694e+17
 3.21137823e+17 9.25112364e+16]


In [210]:
x_right = np.fromfile("filtereddata/right3.dat",dtype=np.float32)
iSampleCount_right = x_right.shape[0]           
t_right = np.linspace(0,iSampleCount_right/iSampleRate,iSampleCount_right)

xFFT_right = np.abs(np.fft.rfft(x_right)/iSampleCount_right)  
xFreqs_right = np.linspace(0, iSampleRate/2, int(iSampleCount_right/2)+1)
print(xFreqs_right)
print(xFFT_right)


[0.00000000e+00 3.26592465e-02 6.53184930e-02 ... 4.99993468e+03
 4.99996734e+03 5.00000000e+03]
[2.13318543e+21 2.38329499e+17 2.29355954e+17 ... 2.32597315e+17
 3.09511405e+17 6.73160087e+16]


In [211]:
x_blink = np.fromfile("filtereddata/blink3.dat",dtype=np.float32)
iSampleCount_blink = x_blink.shape[0]           
t_blink = np.linspace(0,iSampleCount_blink/iSampleRate,iSampleCount_blink)

xFFT_blink = np.abs(np.fft.rfft(x_blink)/iSampleCount_blink)  
xFreqs_blink = np.linspace(0, iSampleRate/2, int(iSampleCount_blink/2)+1)
print(xFreqs_blink)
print(xFFT_blink)

[0.00000000e+00 3.26592465e-02 6.53184930e-02 ... 4.99993468e+03
 4.99996734e+03 5.00000000e+03]
[2.13318736e+21 2.52727185e+17 4.10454837e+17 ... 9.32281293e+16
 1.13379267e+17 2.60143016e+17]


In [212]:
x_doubleblink = np.fromfile("filtereddata/doubleblink3.dat",dtype=np.float32)
iSampleCount_doubleblink = x_doubleblink.shape[0]           
t_doubleblink = np.linspace(0,iSampleCount_doubleblink/iSampleRate,iSampleCount_doubleblink)

xFFT_doubleblink = np.abs(np.fft.rfft(x_doubleblink)/iSampleCount_doubleblink)  
xFreqs_doubleblink = np.linspace(0, iSampleRate/2, int(iSampleCount_doubleblink/2)+1)
print(xFreqs_doubleblink)
print(xFFT_doubleblink)

[0.00000000e+00 3.26592465e-02 6.53184930e-02 ... 4.99993468e+03
 4.99996734e+03 5.00000000e+03]
[2.13310181e+21 1.45497746e+17 5.09474462e+17 ... 4.44359570e+17
 3.28449503e+17 1.60294783e+17]


In [24]:
#SD of training set 
i=1
test = []
while i < 11:
    filename = '/Users/lily/Desktop/data3888/filtereddata/training_2.dat'
    array = np.loadtxt(filename)
    var_test = np.var(array)
    test.append(np.sqrt(var_test))
    i+=1
# range
test_max_sd = max(test)
test_min_sd = min(test)
print(test_max_sd)
print(test_min_sd)
# mean of SD
test_mean_SD = np.mean(test)
print(test_mean_SD)
# sd of sd 
test_sd_SD = np.sqrt(np.var(test))
print(test_sd_SD)

7.368751981481932
7.368751981481932
7.368751981481933
8.881784197001252e-16


In [26]:
iSampleRate = 10000
x_test = np.fromfile("/Users/lily/Desktop/data3888/filtereddata/training_2.dat",dtype=np.float32)
iSampleCount_test = x_test.shape[0]           
t_test = np.linspace(0,iSampleCount_test/iSampleRate,iSampleCount_test)

xFFT_test = np.abs(np.fft.rfft(x_test)/iSampleCount_test)  
xFreqs_test = np.linspace(0, iSampleRate/2, int(iSampleCount_test/2)+1)
print(xFreqs_test)
print(xFFT_test)

[0.00000000e+00 1.33471254e-03 2.66942507e-03 ... 4.99999733e+03
 4.99999867e+03 5.00000000e+03]
[2.13339418e+21 1.92406343e+16 1.17032751e+16 ... 7.32783013e+16
 1.89114314e+16 1.57575044e+15]


In [ ]:
for i in 